<div style="display: flex; background-color: RGB(255,114,0);" >

# PROJET - Cojoden
</div>

<div style="display: flex; background-color: blue;" >

# 0. Référentiel : Développer la gestion des données analytiques d’un projet d’application
</div>

<div style="display: flex; background-color: Green; padding: 7px;" >

### 0.1. Rapport
</div>

- La documentation des sources de données identifiées nécessaires au projet, en fonction des résultats de l’analyse exploratoire.
- Les visualisations synthétisant les caractéristiques du ou des jeux de données.
- Le script de nettoyage des données.
- Les éléments de modélisation des données - schémas, requêtes, etc.
- La procédure de mise en place du système de gestion de base de données analytique.
- Le script d’insertion des données dans le système de gestion de base de données.

<div style="display: flex; background-color: Green; padding: 7px;" >

### 0.2. Soutenance
</div>

- Développer la gestion des données analytiques d’un projet d’application
- La documentation des sources de données identifiées nécessaires au projet, en fonction des résultats de l’analyse exploratoire.
- Les visualisations synthétisant les caractéristiques du ou des jeux de données - screenshot ou via un applicatif.
- La stratégie de nettoyage des données.
- Les éléments de modélisation des données - schémas, requêtes, etc.
- La justification du ou des systèmes de gestion de bases de données analytique adaptés à la modélisation des données.
- La procédure de mise en place du système de gestion de base de données analytique.

<div style="display: flex; background-color: Green; padding: 7px;" >

### 0.3. Démonstration
</div>

La démonstration doit présenter les éléments suivant :
- Le nettoyage des données.
- L’insertion des données dans le système de gestion de base de données

<div style="display: flex; background-color: blue;" >

# 1.La mission
</div>

La Coopérative des Joyeux Denisoviens, la CoJoDen, une agence de voyages engagée vient de vous embaucher pour créer un prototype informatique.

La CoJoDen a pour objectif d'étoffer son offre et monter en gamme en proposant à ses clients des séjours culturels thématiques. Ces séjours seront définis dynamiquement en fonction des souhaits du client.

Par exemple notre nouvelle cliente italienne Mme Gherardini souhaite organiser un petit périple pour découvrir les oeuvres de Gaston Chaissac. Elle ne dispose que d'un weekend et voudrait limiter ses déplacements à deux musées. Si vous regardez dans https://www.pop.culture.gouv.fr/ et vous faites une recherche sur Gaston Chaissac (plusieurs auteurs avec des casses différentes), vous trouverez des oeuvres principalement sur Nantes et Colmar. Vous pourrez proposer un séjour à cette dame en lui indiquant le temps estimé pour aller d'un lieu à l'autre (à vous de trouver un moyen de récupérer le temps de trajet, ce denier point est optionnel dans un premier temps).

A partir d'un extrait la base de donnée Joconde accessible au grand public, vous modéliserez une base de données relationnelle MySQL qui regroupera les informations que vous jugerez nécessaires.

Vous peuplerez la base avec les oeuvres disponibles en nettoyant et en homogénéisant au maximum les infos.
Vous créerez un code python qui servira de démonstrateur sur quelques exemples simples.

### Livrables

- Faites un démonstrateur python de faisabilité avec quelques références (stockage et accès aux
données)
- Au niveau des livrables, voici ce qu'il faudra: Script python du démonstrateur, pdf de modèle de base (PAS DE NOTEBOOK)
Si on a le temps cette semaine vous ferez une présentation de 10 min de votre programme.
Si c'est trop court pour vous, on fera ça lors de notre prochaine semaine de formation

- NoSQL pour la sauvegarde des PDF et images

### Ressources

- https://www.data.gouv.fr/fr/datasets/collections-des-musees-de-france-base-joconde/


Nomenclature des champs :
Ce jeu de données comprend l’intégralité du catalogue collectif des musées de France. Les notices d’objets des musées de France sont de toute nature (archéologie, beaux-arts, ethnologie…).
Le détail de la nomenclature de la liste des champs de la base Joconde est téléchargeable ci-dessous, voir "Pièce jointe".

<div style="display: flex; background-color: blue;" >

# 2.Extraction des données
</div>

<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.1. Chargement des données
</div>

In [1]:
from os import getcwd
from os.path import join
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import csv
import re
import cojoden_extraction as cojex
from IPython.core.display import HTML
from cojoden_functions import convert_string_to_search_string, convert_df_string_to_search_string
from collections import Counter

In [2]:
# ---------------------------------------------------------------------------------------------
#                               MAIN
# ---------------------------------------------------------------------------------------------
verbose = 0

# Récupère le répertoire du programme
file_path = getcwd() + "\\"
data_set_path = file_path + "dataset\\"

data_set_file_name = "cojoden_origine_clean.csv"

print(f"Current execution path : {file_path}")
print(f"Dataset path : {data_set_path}")

Current execution path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\
Dataset path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\


In [3]:
# Chargement et nettoyage général
df_origin = pd.read_csv(join(data_set_path,data_set_file_name), low_memory=False)
display(HTML(df_origin.head().to_html()))

,ref,titre,auteur,annee_debut,annee_fin,domaine,type_oeuvre,materiaux_technique,museo,nom_officiel_musee,latitude,longitude,geo_ville,geo_departement,geo_region,geo_ecole_pays,creation_lieux,largeur_cm,hauteur_cm,profondeur_cm,inscription_precisions,texte
0,00000000001,Le tableau représente au premier plan le Calavon vu vers l'aval,TORINY Gratien Désiré Huet De - (peintre),NaN,NaN,peinture,tableau,"toile, bois (peinture à l'huile)",M0945,musée municipal d'archéologie,43.875736,5.397687,Musées d'Apt,Vaucluse,Provence-Alpes-Côte d'Azur,NaN,Apt,25.0,37.5,0.0,"(français ), En bas à droite : H. de Toriny ; (français ), au revers : Manuscrit au crayon., 'Vue du Pont-Julien à Bonnieux'","Le tableau représente au premier plan le Calavon vu vers l'aval; au bord de la rivière sont arrêtés une charette à deux roues, deux animaux dételés et un personnage. Un peu plus loin, un cavalier (ou une cavalière ) fait boire son cheval dans la rivière. La rive droite de la rivière ressemble à une plage sableuse, plate et large ; elle est dominée par un petit escarpement rocheux. Sur la rive gauche on devine un berger menant son troupeau, alors que la toiture d'une maison se distingue derrière les monticules rocheux. Au second plan, le Pont-Julien lance ses trois arches par-dessus le lit de la rivière. Les deux piles centrales sont fondées sur des massifs qui se distinguent par leur couleur grise dans la masse brune du pont. La montée nord du pont prolonge la ligne du Luberon dont la masse, en arrière-plan, semble enveloppée d'une brume que le peintre a traitée en camaïeu violacé. Un ciel bleu occupant toute la moitié supérieure du tableau est traversé de quelques nuages à frange rosée."
1,00000000002,Panneau de mosaïque dans un cadre en bois,NaN,NaN,NaN,archéologie;céramique;arts décoratifs,mosaïque,"mortier de chaux, pierre indéterminée, colorant(s), pigment(s) artificiel(s) ou dérivé(s, bois (mosaïque)",M0945,musée municipal d'archéologie,43.875736,5.397687,Musées d'Apt,Vaucluse,Provence-Alpes-Côte d'Azur,NaN,NaN,82.0,0.0,0.0,NaN,"Panneau de mosaïque dans un cadre en bois. ; Fragment de mosaïque romaine (pavement de sol) à décor floral et géométrique polychrome, encadré d'entrelacs ; le tout dans un cadre en bois. Fiche ancienne d'André DUMOULIN : 'Fragment important de mosaïque romaine à petits cubes blancs, rouges et noirs. 2 zones : motif ornemental (4 étoiles à 4 pointes) et une fleur ; encadrement d'entrelacs.'"
2,00000000003,Sculpture en calcaire tendre local provenant du domaine du Clos,NaN,NaN,NaN,sculpture,Sculpture en calcaire tendre local provenant du domaine du Clos,calcaire (sculpture),M0945,musée municipal d'archéologie,43.875736,5.397687,Musées d'Apt,Vaucluse,Provence-Alpes-Côte d'Azur,NaN,NaN,0.0,118.0,0.0,Sur l'objet : Inscription manuscrite peinte,"Sculpture en calcaire tendre local provenant du domaine du Clos, occupé en partie par l'actuel jardin public d'APT. Cette sculpture se trouvait auparavant dans le hall de l'Hôtel de Ville. Statue grandeur nature (1,18 m, sans la tête), représentant une déesse-mère, une MATER, assise sur un coussin disposé sur un tabouret à quatre pieds simples, reliés par des traverses delphiniformes et recouvert d'un lourd tissu frangé (hauteur du siège : 45 cm). Les bras reposent sur les cuisses dans une attitude hiératique. Elle est vêtue d'une tunique à manches mi-longues plissées qui recouvre les pieds habillés de chaussures fermées. Un manteau ample, drapé, découvre l'épaule et le bras droit et s'enroule autour du bras gauche ; un pan retombe en plis entre les jambes. Des encoches indiquent la position d'attributs disparus anciennement : la main droite tenait une patère en signe d'offrande et la main gauche repliée (dont l'annulaire est orné d'une bague) semble tenir et relever un pan du manteau tandis que dans le creux du bras reposait vraisemblablement une corne d'abondance. Sur les genoux, dans le creux du manteau, étaient déposés soit des fruits, soit une corbeille remplie de fruits (encoche

In [4]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605157 entries, 0 to 605156
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ref                     605157 non-null  object 
 1   titre                   600532 non-null  object 
 2   auteur                  528887 non-null  object 
 3   annee_debut             263230 non-null  float64
 4   annee_fin               62097 non-null   float64
 5   domaine                 605156 non-null  object 
 6   type_oeuvre             598271 non-null  object 
 7   materiaux_technique     543702 non-null  object 
 8   museo                   605079 non-null  object 
 9   nom_officiel_musee      599077 non-null  object 
 10  latitude                605157 non-null  float64
 11  longitude               605157 non-null  float64
 12  geo_ville               605119 non-null  object 
 13  geo_departement         599077 non-null  object 
 14  geo_region          

<div style="display: flex; background-color: blue;" >

# SEPARATION des données
</div>

In [5]:
run_extraction = 0

<div style="display: flex; background-color: indigo;" >

#### METIERS
</div>

In [6]:
if run_extraction > 0:
    df_metiers= cojex.extract_data(df=df_origin, src_col_name="auteur", dest_col_name="metier", clean_function=cojex.clean_metier, separator="\)", dest_path=data_set_path, dest_file_name='metiers_uniques.csv', verbose=verbose)
else:
    df_metiers = pd.read_csv(join(data_set_path,'cojoden_metiers_uniques.csv'))
print(f"{df_metiers.shape} données chargées ------> {list(df_metiers.columns)}")
df_metiers

(230, 2) données chargées ------> ['search', 'metier']


,search,metier
0,ADAPTATEUR,adaptateur
1,ADMINISTRATEUR,administrateur
2,AFFICHISTE,affichiste
3,AMATEUR,amateur
4,AQUAFORTISTE,aquafortiste
...,...,...
225,TUILIER,tuilier
226,TYPOGRAPHE,typographe
227,VANNIER,vannier
228,VERNISSEUR,vernisseur


<div style="display: flex; background-color: indigo;" >

#### VILLES
</div>

In [7]:
if run_extraction > 0:
    df_villes = cojex.extract_villes(df=df_origin, dest_path=data_set_path, dest_file_name='villes_departement_region_pays.csv',verbose=verbose)
else:
    df_villes = pd.read_csv(join(data_set_path,'cojoden_villes_departement_region_pays.csv'))
    df_villes = df_villes.set_index("id")
print(f"{df_villes.shape} données chargées ------> {list(df_villes.columns)}")

(377, 6) données chargées ------> ['ville_search', 'ville', 'departement', 'region1', 'region2', 'pays']


In [8]:
df_villes[(df_villes['pays'].notna()) & (df_villes['pays']!='France')]

,ville_search,ville,departement,region1,region2,pays
id,,,,,,
50,BONN,Bonn,NaN,NaN,NaN,Allemagne
63,BRUXELLES,Bruxelles,NaN,NaN,NaN,Belgique
115,DUBLIN,Dublin,NaN,NaN,NaN,République d'Irlande
169,LE VATICAN,Le Vatican,NaN,NaN,NaN,Italie
181,LONDRES,Londres,NaN,NaN,NaN,Grande-Bretagne
188,LUXEMBOURG,Luxembourg,NaN,NaN,NaN,Luxembourg
285,ROME,Rome,NaN,NaN,NaN,Italie
296,SAINT DENIS DE LA REUNION,Saint-Denis de La Réunion,Réunion,NaN,NaN,La Réunion
343,TOKYO,Tokyo,NaN,NaN,NaN,Japon


In [9]:
df_villes[(df_villes['departement'].isna())]

,ville_search,ville,departement,region1,region2,pays
id,,,,,,
50,BONN,Bonn,NaN,NaN,NaN,Allemagne
61,BRIANCON,Briançon,NaN,NaN,NaN,France
63,BRUXELLES,Bruxelles,NaN,NaN,NaN,Belgique
73,CASTELLARE DI MERCURIO,Castellare-di-Mercurio,NaN,Corse,NaN,France
115,DUBLIN,Dublin,NaN,NaN,NaN,République d'Irlande
169,LE VATICAN,Le Vatican,NaN,NaN,NaN,Italie
181,LONDRES,Londres,NaN,NaN,NaN,Grande-Bretagne
188,LUXEMBOURG,Luxembourg,NaN,NaN,NaN,Luxembourg
285,ROME,Rome,NaN,NaN,NaN,Italie


<div style="display: flex; background-color: indigo;" >

#### MUSEES
</div>

In [10]:
if run_extraction > 0:
    df_musees = cojex.extract_musees(df=df_origin, df_villes=df_villes, dest_path=data_set_path, dest_file_name='musees.csv', verbose=verbose)
else:
    df_musees = pd.read_csv(join(data_set_path,'cojoden_musees.csv'))
print(f"{df_musees.shape} données chargées ------> {list(df_musees.columns)}")
df_musees

(482, 5) données chargées ------> ['museo', 'nom', 'ville', 'latitude', 'longitude']


,museo,nom,ville,latitude,longitude
0,M0811,musée Boucher de Perthes,1,50.107120,1.833193
1,M0465,musée de l'Ephèbe,2,43.298047,3.476702
2,M0087,musée municipal des beaux-arts,3,44.202895,0.615471
3,M0894,musée Granet,4,43.494315,5.410889
4,M0895,musée de l'atelier de Paul Cézanne,4,43.537816,5.446601
...,...,...,...,...,...
477,M1008,musée départemental de la Révolution française,374,45.087530,5.773234
478,M1137,musée Mainssieux,375,45.356047,5.579652
479,M0191,musée Zervos,364,47.464133,3.743610
480,M0022,musée Westercamp,376,49.038815,7.944344


<div style="display: flex; background-color: indigo;" >

#### AUTEURS
</div>

In [11]:
df_origin.columns

Index(['ref', 'titre', 'auteur', 'annee_debut', 'annee_fin', 'domaine',
       'type_oeuvre', 'materiaux_technique', 'museo', 'nom_officiel_musee',
       'latitude', 'longitude', 'geo_ville', 'geo_departement', 'geo_region',
       'geo_ecole_pays', 'creation_lieux', 'largeur_cm', 'hauteur_cm',
       'profondeur_cm', 'inscription_precisions', 'texte'],
      dtype='object')

In [12]:
if run_extraction > 0:
    df_artistes = cojex.extract_artistes(df=df_origin, dest_path=data_set_path, dest_file_name='artistes.csv', verbose=verbose)
else:
    df_artistes = pd.read_csv(join(data_set_path,'cojoden_artistes.csv'))
    df_artistes = df_artistes.set_index("id")
print(f"{df_artistes.shape} données chargées ------> {list(df_artistes.columns)}")
df_artistes

(46235, 3) données chargées ------> ['nom_search', 'nom_naissance', 'dit']


,nom_search,nom_naissance,dit
id,,,
2,A B,A B,NaN
3,A B ET CIE,A B & Cie,NaN
4,A B R,A B R,NaN
5,A BORD ET CIE,A. Bord et Cie,NaN
6,A C,A C,NaN
...,...,...,...
46232,ZYLBERBERG ADEK,ZYLBERBERG Adek,NaN
46233,ZYLBERBERG MARIE,ZYLBERBERG Marie,NaN
46234,ZYLVELT ADAM VAN,ZYLVELT Adam Van,NaN


<div style="display: flex; background-color: indigo;" >

#### OEUVRES
</div>

In [13]:
if run_extraction > 0:
    df_oeuvres = cojex.extract_oeuvres(df=df_origin, df_villes=df_villes, dest_path=data_set_path, dest_file_name='oeuvres.csv', verbose=0)
else:
    df_oeuvres = pd.read_csv(join(data_set_path,'cojoden_oeuvres.csv'), low_memory=False)
print(f"{df_oeuvres.shape} données chargées ------> {list(df_oeuvres.columns)}")
df_oeuvres

(605157, 13) données chargées ------> ['ref', 'titre', 'type', 'domaine', 'annee_debut', 'annee_fin', 'lieux_conservation', 'largeur_cm', 'hauteur_cm', 'profondeur_cm', 'creation_lieux', 'inscriptions', 'texte']


,ref,titre,type,domaine,annee_debut,annee_fin,lieux_conservation,largeur_cm,hauteur_cm,profondeur_cm,creation_lieux,inscriptions,texte
0,00000000001,Le tableau représente au premier plan le Calav...,tableau,peinture,NaN,NaN,M0945,25.0,37.5,0.0,16,"(français ), En bas à droite : H. de Toriny ; ...",Le tableau représente au premier plan le Calav...
1,00000000002,Panneau de mosaïque dans un cadre en bois,mosaïque,archéologie;céramique;arts décoratifs,NaN,NaN,M0945,82.0,0.0,0.0,0,NaN,Panneau de mosaïque dans un cadre en bois. ; F...
2,00000000003,Sculpture en calcaire tendre local provenant d...,Sculpture en calcaire tendre local provenant d...,sculpture,NaN,NaN,M0945,0.0,118.0,0.0,0,Sur l'objet : Inscription manuscrite peinte,Sculpture en calcaire tendre local provenant d...
3,00000000004,Sculpture monilithique en ronde-bosse,calcaire (sculpture) Sculpture monilithique en...,sculpture,NaN,NaN,M0945,32.0,50.0,0.0,0,Sur l'objet : Inscription manuscrite à l'encre...,"Sculpture monilithique en ronde-bosse, dans un..."
4,00000000005,statuette Marbre blanc,statuette,sculpture,NaN,NaN,M0945,14.0,28.0,0.0,0,NaN,Marbre blanc (marbre de Carrare ). Traces de p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
605152,M7051022442,plaque de verre Portrait de la jeune femme aux...,plaque de verre,ethnologie;photographie,NaN,NaN,M7051,9.0,0.0,12.0,0,NaN,Portrait de la jeune femme aux cheveux courts ...
605153,M7051022447,plaque de verre Notre personnage à cheveux cou...,plaque de verre,ethnologie;photographie,NaN,NaN,M7051,9.0,0.0,12.0,0,NaN,Notre personnage à cheveux courts porte cette ...
605154,M7051022448,plaque de verre Photo : jeune femme cheveux co...,plaque de verre,ethnologie;photographie,NaN,NaN,M7051,9.0,0.0,12.0,0,NaN,Photo : jeune femme cheveux courts a enlevé so...
605155,M7051030233,vue stéréoscopique Une jeune enfant en photo p...,vue stéréoscopique,ethnologie;photographie,NaN,NaN,M7051,4.5,0.0,11.0,0,NaN,Une jeune enfant en photo porte un petit béret...


<div style="display: flex; background-color: indigo;" >

#### CREER
</div>

In [14]:
if run_extraction > 0:
    df_creation_oeuvres = cojex.extract_creation_oeuvres(df=df_origin, df_artistes=df_artistes, dest_path=data_set_path, dest_file_name='creation_oeuvres.csv', verbose=verbose)
else:
    df_creation_oeuvres = pd.read_csv(join(data_set_path,'cojoden_creation_oeuvres.csv'), low_memory=False)
print(f"{df_creation_oeuvres.shape} données chargées ------> {list(df_creation_oeuvres.columns)}")
df_creation_oeuvres

(639238, 3) données chargées ------> ['oeuvre', 'artiste', 'role']


,oeuvre,artiste,role
0,00000000001,42739,PEINTRE
1,00000000013,26677,NaN
2,00000000014,35734,SCULPTEUR
3,00000041950,24732,VERRIER
4,00000041951,24732,CERAMISTE
...,...,...,...
639233,M0381001687,45687,GRAVEUR
639234,M0381001687,44625,DESSINATEUR
639235,M0381001687,34838,DESSINATEUR
639236,M0381001687,23087,DESSINATEUR


<div style="display: flex; background-color: indigo;" >

#### COMPOSER
</div>

In [15]:
run_extraction = 1

In [16]:
df2 = df_origin.loc[df_origin['materiaux_technique'].notna(), 'materiaux_technique'].str.split(r",", expand=True)
df3 = pd.merge(df_origin[['ref','materiaux_technique']],df2, left_index=True, right_index=True)
df4 = cojex.transpose_df_after_split_extend(df3, cols_to_keep=['ref','materiaux_technique'], verbose=1)
df4

[transpose_df_after_split_extend]	 INFO : (543702, 3) on origin df and [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] to proceed
[transpose_df_after_split_extend]	 INFO : (867665, 3) at the end and Index(['ref', 'materiaux_technique', 0], dtype='object')


,ref,materiaux_technique,0
0,00000000001,"toile, bois (peinture à l'huile)",toile
0,00000000001,"toile, bois (peinture à l'huile)",bois (peinture à l'huile)
1,00000000002,"mortier de chaux, pierre indéterminée, coloran...",mortier de chaux
1,00000000002,"mortier de chaux, pierre indéterminée, coloran...",pierre indéterminée
1,00000000002,"mortier de chaux, pierre indéterminée, coloran...",colorant(s)
...,...,...,...
605152,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine
605153,M7051022447,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine
605154,M7051022448,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine
605155,M7051030233,verre (photographie),verre (photographie)


In [21]:
if run_extraction > 0:
    df_materiaux, df_compose = cojex.extract_oeuvre_materiaux(df=df_origin, dest_path=data_set_path, dest_file_name='materiaux_techniques.csv', verbose=verbose)
else:
    df_materiaux = pd.read_csv(join(data_set_path,'cojoden_materiaux_techniques.csv'))
    df_materiaux = df_materiaux.set_index("id")

    df_compose = pd.read_csv(join(data_set_path,'cojoden_materiaux_techniques_compose.csv'))

print(f"{df_materiaux.shape} données chargées ------> {list(df_materiaux.columns)}")
print(f"{df_compose.shape} données chargées ------> {list(df_compose.columns)}")
df_materiaux

[extract_oeuvre_compose_mat]	 INFO : (8462, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\materiaux_techniques.csv
[extract_oeuvre_compose_mat]	 INFO : (1247151, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\materiaux_techniques_compose.csv
(8462, 3) données chargées ------> ['id', 'mat_search', 'materiaux_technique']
(1247151, 2) données chargées ------> ['oeuvre', 'materiaux']


,id,mat_search,materiaux_technique
0,1,1 COULEUR,1 couleur
1,2,10,(10)
2,3,12,12
3,4,13,13
4,5,14,14
...,...,...,...
8457,8458,ZINC ZINCOGRAPHIE,zinc (zincographie
8458,8459,ZINCOGRAPHIE,zincographie (?
8459,8460,ZINCOGRAPHIE COLORIE,zincographie (colorié)
8460,8461,ZINCOGRAPHIE COLORIE AU POCHOIR,zincographie (colorié au pochoir)


In [22]:
df_compose.shape

(1247151, 2)

In [57]:
# Il faut construire une DF avec les métriaux
df3 = df_origin.loc[df_origin['materiaux_technique'].notna(), 'materiaux_technique'].str.split(",", expand=True)
# On remet la référence de l'oeuvre, pour ça on merge les df sur l'index.
df4 = pd.merge(df_origin[["ref", 'materiaux_technique']],df3, left_index=True, right_index=True)

df5 = cojex.transpose_df_after_split_extend(df4, cols_to_keep=["ref", 'materiaux_technique'] , verbose=verbose-1)
df5 = df5.sort_values('ref')
df5 = df5.reset_index()
df6 = df5.rename(columns={'materiaux_technique':'mat_src', 0:'materiaux_technique'})
df2 = df6[["ref", 'mat_src', 'materiaux_technique']]
df2 = df2.drop_duplicates()

In [58]:
df3

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,toile,bois (peinture à l'huile),None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,mortier de chaux,pierre indéterminée,colorant(s),pigment(s) artificiel(s) ou dérivé(s,bois (mosaïque),None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,calcaire (sculpture),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,calcaire (sculpture),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,marbre,peinture (sculpture),None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605152,plaque de verre (bromure d'argent);gélatine,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
605153,plaque de verre (bromure d'argent);gélatine,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
605154,plaque de verre (bromure d'argent);gélatine,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
605155,verre (photographie),None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [59]:
df4

,ref,materiaux_technique,0,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31
0,00000000001,"toile, bois (peinture à l'huile)",toile,bois (peinture à l'huile),None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,00000000002,"mortier de chaux, pierre indéterminée, coloran...",mortier de chaux,pierre indéterminée,colorant(s),pigment(s) artificiel(s) ou dérivé(s,bois (mosaïque),None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,00000000003,calcaire (sculpture),calcaire (sculpture),None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,00000000004,calcaire (sculpture),calcaire (sculpture),None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,00000000005,"marbre, peinture (sculpture)",marbre,peinture (sculpture),None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605152,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
605153,M7051022447,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
605154,M7051022448,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
605155,M7051030233,verre (photographie),verre (photographie),None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [68]:
df5[df5["ref"]=="M7051022442"]

,index,ref,materiaux_technique,0
2118504,867660,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent)
2118505,867660,M7051022442,plaque de verre (bromure d'argent);gélatine,gélatine
2118506,867660,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine


In [69]:
df6[df6["ref"]=="M7051022442"]

,index,ref,mat_src,materiaux_technique
2118504,867660,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent)
2118505,867660,M7051022442,plaque de verre (bromure d'argent);gélatine,gélatine
2118506,867660,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine


In [70]:
df7[df7["ref"]=="M7051022442"]

,ref,mat_src,materiaux_technique
2118504,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent)
2118505,M7051022442,plaque de verre (bromure d'argent);gélatine,gélatine
2118506,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine


In [63]:
# Il faut construire une DF avec les métriaux
df3 = df2.loc[df2['materiaux_technique'].notna(), 'materiaux_technique'].str.split(";", expand=True)
# On remet la référence de l'oeuvre, pour ça on merge les df sur l'index.
df4 = pd.merge(df2,df3, left_index=True, right_index=True)

df5 = transpose_df_after_split_extend(df4, cols_to_keep=["ref", 'materiaux_technique'] , verbose=verbose-1)
df5 = df5.sort_values('ref')
df5 = df5.reset_index()
df6 = df5.rename(columns={'materiaux_technique':'mat_src', 0:'materiaux_technique'})
df7 = df6[["ref", 'mat_src', 'materiaux_technique']]
df7 = df7.drop_duplicates()

In [65]:
df3

,0,1,2,3,4,5,6,7,8,9,10,11
0,toile,None,None,None,None,None,None,None,None,None,None,None
1,bois (peinture à l'huile),None,None,None,None,None,None,None,None,None,None,None
2,colorant(s),None,None,None,None,None,None,None,None,None,None,None
3,pigment(s) artificiel(s) ou dérivé(s,None,None,None,None,None,None,None,None,None,None,None
4,bois (mosaïque),None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
867660,plaque de verre (bromure d'argent),gélatine,None,None,None,None,None,None,None,None,None,None
867661,plaque de verre (bromure d'argent),gélatine,None,None,None,None,None,None,None,None,None,None
867662,plaque de verre (bromure d'argent),gélatine,None,None,None,None,None,None,None,None,None,None
867663,verre (photographie),None,None,None,None,None,None,None,None,None,None,None


In [66]:
df4

,ref,mat_src,materiaux_technique,0,1,2,3,4,5,6,7,8,9,10,11
0,00000000001,"toile, bois (peinture à l'huile)",toile,toile,None,None,None,None,None,None,None,None,None,None,None
1,00000000001,"toile, bois (peinture à l'huile)",bois (peinture à l'huile),bois (peinture à l'huile),None,None,None,None,None,None,None,None,None,None,None
2,00000000002,"mortier de chaux, pierre indéterminée, coloran...",colorant(s),colorant(s),None,None,None,None,None,None,None,None,None,None,None
3,00000000002,"mortier de chaux, pierre indéterminée, coloran...",pigment(s) artificiel(s) ou dérivé(s,pigment(s) artificiel(s) ou dérivé(s,None,None,None,None,None,None,None,None,None,None,None
4,00000000002,"mortier de chaux, pierre indéterminée, coloran...",bois (mosaïque),bois (mosaïque),None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867660,M7051022442,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent),gélatine,None,None,None,None,None,None,None,None,None,None
867661,M7051022447,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent),gélatine,None,None,None,None,None,None,None,None,None,None
867662,M7051022448,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent),gélatine,None,None,None,None,None,None,None,None,None,None
867663,M7051030233,verre (photographie),verre (photographie),verre (photographie),None,None,None,None,None,None,None,None,None,None,None


In [67]:
df5

,index,ref,materiaux_technique,0
0,0,00000000001,toile,toile
1,1,00000000001,bois (peinture à l'huile),"toile, bois (peinture à l'huile)"
2,0,00000000001,toile,"toile, bois (peinture à l'huile)"
3,1,00000000001,bois (peinture à l'huile),bois (peinture à l'huile)
4,2,00000000002,colorant(s),colorant(s)
...,...,...,...,...
2118512,867662,M7051022448,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent)
2118513,867663,M7051030233,verre (photographie),verre (photographie)
2118514,867663,M7051030233,verre (photographie),verre (photographie)
2118515,867664,M7051030569,photographie (verre),photographie (verre)


In [64]:
df7

,ref,mat_src,materiaux_technique
0,00000000001,toile,toile
1,00000000001,bois (peinture à l'huile),"toile, bois (peinture à l'huile)"
2,00000000001,toile,"toile, bois (peinture à l'huile)"
3,00000000001,bois (peinture à l'huile),bois (peinture à l'huile)
4,00000000002,colorant(s),colorant(s)
...,...,...,...
2118510,M7051022448,plaque de verre (bromure d'argent);gélatine,gélatine
2118511,M7051022448,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent);gélatine
2118512,M7051022448,plaque de verre (bromure d'argent);gélatine,plaque de verre (bromure d'argent)
2118513,M7051030233,verre (photographie),verre (photographie)


In [35]:
def extract_oeuvre_materiaux_1(df, dest_path, dest_file_name='materiaux.csv', verbose=0):
    short_name = "extract_oeuvre_compose_mat"
    df1 = df[['ref','materiaux_technique']]
    if verbose > 0:
        print(f"[{short_name}]\t INFO : {df1.shape} on origin df")
    
    df1 = df1[df1['materiaux_technique'].notna()]
    df1 = df1.sort_values('materiaux_technique')
    df1 = df1.drop_duplicates()
    if verbose > 0:
        print(f"[{short_name}]\t INFO : {df1.shape} without na and duplicates")

    df2 = df1.copy()

    # Séparation des matériaux
    for sep in [',', ';', '/', '.', ':']:
        # Il faut construire une DF avec les métriaux
        df3 = df2.loc[df2['materiaux_technique'].notna(), 'materiaux_technique'].str.split(sep, expand=True)
        # On remet la référence de l'oeuvre, pour ça on merge les df sur l'index.
        df4 = pd.merge(df2,df3, left_index=True, right_index=True)
        
        df5 = cojex.transpose_df_after_split_extend(df4, cols_to_keep=["ref", 'materiaux_technique'] , verbose=verbose-1)
        df5 = df5.rename(columns={'materiaux_technique':'mat_src', 0:'materiaux_technique'})
        df2 = df5[["ref", 'mat_src', 'materiaux_technique']]
        df2 = df2.drop_duplicates()

    # nettoyer les chaines
    df2.loc[df2["materiaux_technique"].notna(),"materiaux_technique"] = df2.loc[df2["materiaux_technique"].notna(),"materiaux_technique"].apply(lambda x:cojonet.clean_text(x, verbose=verbose-1))
    df2 = df2.drop_duplicates()
    
    # Création du nom pour les recherches
    df_composer = df2.copy()
    df_composer["mat_search"] = df_composer['materiaux_technique']
    df_composer = convert_df_string_to_search_string(input_df=df_composer, col_name="mat_search")
    df_composer['materiaux_technique'] = df_composer['materiaux_technique'].str.lower()

    if verbose > 0:
        print(f"[{short_name}]\t INFO : {df_composer.shape}")
    df_composer = df_composer.sort_values(["mat_search"])
    df_composer = df_composer.drop_duplicates(['ref','mat_search'], keep='first')
    if verbose > 0:
        print(f"[{short_name}]\t INFO : {df_composer.shape}")
    
    df_composer = df_composer[(df_composer["mat_search"].notna())&(df_composer["mat_search"]!="")&(df_composer["mat_search"].str.len()>1)]

    return df_composer


In [34]:
df_compose = extract_oeuvre_materiaux_1(df=df_origin, dest_path=data_set_path, dest_file_name='materiaux_techniques.csv', verbose=verbose)
df_compose

,ref,mat_src,materiaux_technique,mat_search
100338,00440002857,impression au rouleau,1 couleur,1 COULEUR
100139,00440001913,contrefond impression mécanique : 1 couleur,1 couleur,1 COULEUR
100683,00440008820,"papier, fond mat;contrefond impression mécaniq...",1 couleur,1 COULEUR
100683,00440008820,"papier, fond mat;contrefond impression mécaniq...","1 couleur;impression à la planche, nombre de c...",1 COULEUR IMPRESSION A LA PLANCHE NOMBRE DE CO...
100139,00440001913,impression au rouleau,"1 couleur;impression au rouleau, en relief , n...",1 COULEUR IMPRESSION AU ROULEAU EN RELIEF NOMB...
...,...,...,...,...
525019,M0170002906,"oxyde de fer, manganèse, zinc (moulé)",zinc (moulé),ZINC MOULE
525018,M0170002905,NaN,zinc (moulé),ZINC MOULE
120047,01550011370,"papier, zincographie",zincographie,ZINCOGRAPHIE
516272,M0010008576,"papier (encre noire, zincographie, cliché typo...",zincographie,ZINCOGRAPHIE


In [33]:
df_compose[df_compose['ref']=='M0170002906']

,ref,mat_src,materiaux_technique,mat_search
525019,M0170002906,"oxyde de fer, manganèse, zinc (moulé)",manganèse,MANGANESE
525019,M0170002906,"oxyde de fer, manganèse, zinc (moulé)",zinc (moulé),ZINC MOULE


In [29]:
if run_extraction > 0:
    df_materiaux, df_compose = extract_oeuvre_materiaux(df=df_origin, dest_path=data_set_path, dest_file_name='materiaux_techniques.csv', verbose=verbose)
else:
    df_materiaux = pd.read_csv(join(data_set_path,'cojoden_materiaux_techniques.csv'))
    df_materiaux = df_materiaux.set_index("id")

    df_compose = pd.read_csv(join(data_set_path,'cojoden_materiaux_techniques_compose.csv'))

print(f"{df_materiaux.shape} données chargées ------> {list(df_materiaux.columns)}")
print(f"{df_compose.shape} données chargées ------> {list(df_compose.columns)}")
df_materiaux

[extract_oeuvre_compose_mat]	 INFO : (10580, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\materiaux_techniques.csv
[extract_oeuvre_compose_mat]	 INFO : (791682, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\materiaux_techniques_compose.csv
(10580, 3) données chargées ------> ['id', 'mat_search', 'materiaux_technique']
(791682, 2) données chargées ------> ['oeuvre', 'materiaux']


,id,mat_search,materiaux_technique
0,1,1,1
1,2,1 COULEUR,1 couleur
2,3,1 COULEUR IMPRESSION A LA PLANCHE NOMBRE DE CO...,"1 couleur;impression à la planche, nombre de c..."
3,4,1 COULEUR IMPRESSION AU ROULEAU EN RELIEF NOMB...,"1 couleur;impression au rouleau, en relief , n..."
4,5,1 IMPRESSION A LA PLANCHE NOMBRE DE COULEURS,"1;impression à la planche, nombre de couleurs"
...,...,...,...
10575,10576,ZINC FORGE,zinc (forgé)
10576,10577,ZINC MODELE,zinc (modelé)
10577,10578,ZINC MOULE,zinc (moulé)
10578,10579,ZINCOGRAPHIE,zincographie


In [27]:
if run_extraction > 0:
    df_materiaux, df_compose = extract_oeuvre_materiaux(df=df_origin, dest_path=data_set_path, dest_file_name='materiaux_techniques.csv', verbose=verbose)
else:
    df_materiaux = pd.read_csv(join(data_set_path,'cojoden_materiaux_techniques.csv'))
    df_materiaux = df_materiaux.set_index("id")

    df_compose = pd.read_csv(join(data_set_path,'cojoden_materiaux_techniques_compose.csv'))

print(f"{df_materiaux.shape} données chargées ------> {list(df_materiaux.columns)}")
print(f"{df_compose.shape} données chargées ------> {list(df_compose.columns)}")
df_materiaux

[extract_oeuvre_compose_mat]	 INFO : (10584, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\materiaux_techniques.csv
[extract_oeuvre_compose_mat]	 INFO : (991673, 2) données sauvegardées in ------> c:\Users\User\WORK\workspace-ia\PROJETS\projet_cojoden_avance\dataset\materiaux_techniques_compose.csv
(10584, 3) données chargées ------> ['id', 'mat_search', 'materiaux_technique']
(991673, 2) données chargées ------> ['oeuvre', 'materiaux']


,id,mat_search,materiaux_technique
0,1,,?
1,2,,)
2,3,,
3,4,1,1
4,5,1 COULEUR,1 couleur
...,...,...,...
10579,10580,ZINC MODELE,zinc (modelé)
10580,10581,ZINC MOULE,zinc (moulé)
10581,10582,ZINCOGRAPHIE,zincographie
10582,10583,ZINCOGRAPHIE,zincographie)


<div style="display: flex; background-color: indigo;" >

#### DOMAINE
</div>

Suite à une première analyse, il n'y a pas d'intérêt à traiter les domaines.

In [85]:
df_origin['domaine'].value_counts()

dessin                                                                                235107
peinture                                                                               66689
estampe                                                                                38680
sculpture                                                                              30266
photographie                                                                           13482
                                                                                       ...  
industrie lithique;industrie osseuse;agriculture - élevage;néolithique;archéologie         1
peinture;philatélie                                                                        1
horlogerie;ethnologie;métrologie - instruments de mesure;vie militaire                     1
sculpture;Afrique;ethnologie                                                               1
ethnologie;artisanat - industrie;vie militaire                        

In [87]:
df_clean_na.columns

Index(['ref', 'pop_coordonnees', 'auteur', 'bibl', 'commentaires',
       'type_oeuvre', 'description', 'dimensions', 'domaine',
       'geo_departement', 'geo_ecole_pays', 'hist', 'creation_lieux',
       'lieux_conservation', 'geo_pays_region_ville', 'creation_millesime',
       'museo', 'nom_officiel_musee', 'auteur_precisions', 'creation_periode',
       'inscription_precisions', 'sujet_precisions', 'geo_region', 'sujet',
       'materiaux_technique', 'titre', 'geo_ville', 'texte'],
      dtype='object')

In [88]:
df_domaine = df_clean_na[['domaine']]
df_domaine["search_domaine"] = df_domaine['domaine']
df_domaine = convert_df_string_to_search_string(input_df=df_domaine, col_name="search_domaine", stop_word_to_remove=[])
df_domaine["search_domaine"].value_counts()

C:\Users\User\AppData\Local\Temp\ipykernel_57960\10413957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domaine["search_domaine"] = df_domaine['domaine']


DESSIN                                                                              235107
PEINTURE                                                                             66689
ESTAMPE                                                                              38680
SCULPTURE                                                                            30266
PHOTOGRAPHIE                                                                         13482
                                                                                     ...  
INDUSTRIE LITHIQUE INDUSTRIE OSSEUSE AGRICULTURE ELEVAGE NEOLITHIQUE ARCHEOLOGIE         1
PEINTURE PHILATELIE                                                                      1
HORLOGERIE ETHNOLOGIE METROLOGIE INSTRUMENTS DE MESURE VIE MILITAIRE                     1
SCULPTURE AFRIQUE ETHNOLOGIE                                                             1
ETHNOLOGIE ARTISANAT INDUSTRIE VIE MILITAIRE                                             1

<div style="display: flex; background-color: indigo;" >

#### PERIODES
</div>

In [431]:
# ser_per=pd.Series(", ".join(df_clean['creation_periode'].dropna()).split(";"))
# df_periode1 = pd.DataFrame([ser_per]).T
# print(df_periode1.shape)
# df_periode1 = df_periode1.drop_duplicates()
df_periode2 = df_clean['creation_periode'].dropna().str.split(r";", expand=True)
print(df_periode2.shape)
df_periode2 = df_periode2.drop_duplicates()
print(df_periode2.shape)
df_periode2.head()

(482266, 11)
(1504, 11)


,0,1,2,3,4,5,6,7,8,9,10
0,4e quart 19e siècle,1er quart 20e siècle,None,None,None,None,None,None,None,None,None
2,1ère moitié 20e siècle,None,None,None,None,None,None,None,None,None,None
3,3e quart 19e siècle,None,None,None,None,None,None,None,None,None,None
5,2e quart 19e siècle,None,None,None,None,None,None,None,None,None,None
10,18e siècle,None,None,None,None,None,None,None,None,None,None


In [432]:
df_periode2[df_periode2[10].notna()]

,0,1,2,3,4,5,6,7,8,9,10
6482,1ère moitié 2e siècle av JC,1er siècle av JC,1er siècle,2e siècle,3e siècle,4e siècle,5e siècle,6e siècle,7e siècle,8e siècle,1ère moitié 9e siècle


In [433]:
periodes = set(df_periode2.loc[df_periode2[0].notna(), 0].unique())
periodes

{'1000e millénaire av JC',
 '10e siècle',
 '10e siècle av JC',
 '11e siècle',
 '11e siècle av JC',
 '12e siècle',
 '12e siècle av JC',
 '13e siècle',
 '13e siècle av JC',
 '14e siècle',
 '14e siècle av JC',
 '15e siècle',
 '15e siècle av JC',
 '16e siècle',
 '16e siècle av JC',
 '16eÂ\xa0siècle',
 '16ème siècle',
 '17 e siècle',
 '17e siècle',
 '17e siècle av JC',
 '17è Siècle',
 '17ème siècle',
 '17ème siècle, 2ème quart',
 '18e siècle',
 '18e siècle av JC',
 '18eme siècle',
 '19e',
 '19e siècle',
 '19e siècle (1er quart)',
 '19e siècle (1ère moitié)',
 '19e siècle (3e quart)',
 '19e siècle (4e quart)',
 '19e siècle av JC',
 '19è siècle',
 '19ème siècle 1er quart',
 '19ème siècle 1ère moitié',
 '19ème siècle 3ème quart',
 '19ème siècle 4ème quart',
 '1e quart 18e siècle',
 '1e quart 20e siècle',
 '1e siècle av JC',
 '1er millénaire av JC',
 '1er quart  19e siècle',
 '1er quart  20e siècle',
 '1er quart 10e siècle',
 '1er quart 10e siècle av JC',
 '1er quart 11e siècle',
 '1er quart 12

In [434]:
for i in range(1, 11):
    print(f"{i-1} - {len(periodes)}")
    for val in df_periode2.loc[df_periode2[i].notna(), i].unique():
        periodes.add(val)

print(f"{i-1} - {len(periodes)}")
periodes

0 - 356
1 - 401
2 - 407
3 - 409
4 - 411
5 - 413
6 - 413
7 - 413
8 - 413
9 - 413
9 - 413


{'(?)',
 '1000e millénaire av JC',
 '10e siècle',
 '10e siècle av JC',
 '11e siècle',
 '11e siècle av JC',
 '12e siècle',
 '12e siècle av JC',
 '13e siècle',
 '13e siècle av JC',
 '14e siècle',
 '14e siècle av JC',
 '15e siècle',
 '15e siècle av JC',
 '16e siècle',
 '16e siècle av JC',
 '16eÂ\xa0siècle',
 '16ème siècle',
 '17 e siècle',
 '17e siècle',
 '17e siècle av JC',
 '17è Siècle',
 '17ème siècle',
 '17ème siècle, 2ème quart',
 '18e siècle',
 '18e siècle av JC',
 '18eme siècle',
 '19e',
 '19e siècle',
 '19e siècle (1er quart)',
 '19e siècle (1ère moitié)',
 '19e siècle (3e quart)',
 '19e siècle (4e quart)',
 '19e siècle av JC',
 '19e siècle et',
 '19è siècle',
 '19ème siècle 1er quart',
 '19ème siècle 1ère moitié',
 '19ème siècle 3ème quart',
 '19ème siècle 4ème quart',
 '1e quart 18e siècle',
 '1e quart 19e siècle',
 '1e quart 1er siècle',
 '1e quart 20e siècle',
 '1e siècle av JC',
 '1er millénaire av JC',
 '1er quart   20e siècle',
 '1er quart  18e siècle',
 '1er quart  19e siè

In [170]:
ser_auteurs=pd.Series(", ".join(df_auteur_origin["auteur"].dropna()).split(";"))
df_auteurs1 = pd.DataFrame([ser_auteurs]).T

ser_auteurs=pd.Series(", ".join(df_auteurs1[0].dropna()).split(", "))
df_auteurs = pd.DataFrame([ser_auteurs]).T
df_auteurs = df_auteurs.rename(columns={0:'name'})
df_auteurs.head()

,name
0,LE GULUCHE Joseph Marie
1,THOMSEN Constant
2,FJERDINGSTAD Christian (orfèvre)
3,DUTHOIT Louis
4,DUTHOIT Aimé


In [171]:
print(df_auteurs.shape)
df_auteurs = df_auteurs.drop_duplicates()
print(df_auteurs.shape)

(136322, 1)
(70115, 1)


In [206]:
temp = df_auteurs['name'].str.split(r" \(", expand=True)
temp.head()

,0,1,2,3
0,LE GULUCHE Joseph Marie,None,None,None
1,THOMSEN Constant,None,None,None
2,FJERDINGSTAD Christian,orfèvre),None,None
3,DUTHOIT Louis,None,None,None
4,DUTHOIT Aimé,None,None,None


In [207]:
temp[1].value_counts()

graveur)                    3158
peintre)                    3058
d'après)                    2772
éditeur)                    2576
dessinateur)                2553
                            ... 
LANTZ                          1
1791-1849)                     1
1720-1789)                     1
auteur du modèle))             1
République démocratique)       1
Name: 1, Length: 825, dtype: int64

In [208]:
temp[2].value_counts()

peintre)                   83
dessinateur)               69
émetteur)                  41
?)                         36
d'après)                   28
                           ..
orfèvre)                    1
écrivain))                  1
sculpteur                   1
LEGROS D'ANIZY              1
d'après la peinture de)     1
Name: 2, Length: 82, dtype: int64

In [209]:
temp[3].value_counts()

auteur du modèle)     1
auteur du modèle))    1
auteur du décor))     1
?)                    1
Name: 3, dtype: int64

In [210]:
temp[temp[3].notna()]

,0,1,2,3
1482,LEGROS D'ANIZY,graveur),manufacture de Creil,auteur du modèle)
3019,FAIENCERIES DE CREIL & MONTEREAU,attribué à),FAIENCERIES DE CREIL & MONTEREAU,auteur du modèle))
62203,LEGROS D'ANIZY,graveur),LEGROS D'ANIZY,auteur du décor))
73363,VALENTINIEN IER,FLAVIUS VALENTINIANUS),émetteur),?)


In [413]:
temp = df_auteurs['name'].str.split(r" \(", expand=True)

to_replace_nan = ["d'après", "entourage de", 'née','né', "d'après le dessin de", 
                "suite de", "inspiré par", 'attribué à', 'population', "d'après la peinture de",
                "copie d'après", "amateur", "Collectivité", "usine", "école", "manufacture de Creil",
                "fabrique", "FAIENCERIES DE CREIL & MONTEREAU", "chantier naval", "attribué",
                "style", "attribué à", "dit", "atelier de", "d'après", "attribué Ã "]

to_replace_nan_contains = ["attribué", "manufacture", "patronyme", "1809-", "manière de", "fours de", "Attribué à", 'ateliers', 
'autre attribution', 'FAUX','dit,école de', 'arcades du casino', 'style', 'attr. à', 'faux', 'ECOLE DE','ATTRIBUE','d¹après',
'd’après', 'bar-tabac', 'bazar', 'Attribué'
'établissement', 'dit','atelier de la famille', "d'après la gravure de", 'atelier indéterminé',"D\'","h.",'R M','Cherbourg 1856 - Paris 1913',
'italie','imitation de','école de berry','inspiré par','atelier artistique','?,école','anonyme','laboratoire','groupe de rock','atelier ,','actif vers 1787',
'collection f','autour de','commencé par','terminé par','atelier,?','rue de la paix paris','?,dit','attibué à','ancienne attribution',
'd\x92après','école des','ancienne maison','09',
'pour les fleurs','famillle','pour les personnages',
]
 
to_replace_by = {
        "\)"                        : "",
        '\?'                        : "",
        "couturière"                :"couturier",
        'sculptrice'                :"sculpteur",
        "sculpeur"                  :"sculpteur",
        'sculpteur-repareur'        :"sculpteur, répareur",
        "Editeur"                   :"éditeur",
        'M.-D - EDITEUR - PARIS'    :"éditeur",
        'éditeurPAITL. S. éditeur'  :"éditeur",
        "éditeur d'après CLOUET"    :"éditeur",
        'édition Artistique'        :"éditeur",
        'éditions de luxe'          :"éditeur",
        'éditeur(?'                 :"éditeur",
        'éditeur,?'                 :"éditeur",
        'ouvrier '                  :'',
        'ENGINEERS Ltd.'            :'ingénieur',
        'établissements photographiques':'photographe',
        'compagnie alsacienne des arts photomecaniques':'photographe photomecaniques',
        'photographe,'              :'photographe',
        'lithographe DURAND H'      :'lithographe',
        'composition'               :'compositeur',
        'dit(caricaturiste'         :'caricaturiste',
        "graveur d'après CLOUET"    :"graveur",
        'graveur BASSET'            :'graveur',
        "graveur d'après"           :'graveur',
        'Libraire'                  :'libraire',
        # 'libraire,éditeur'          :'libraire',
        # 'imprimeur,éditeur'         :'imprimeur',
        '18...-18 imprimeur'        :'imprimeur',
        'imprimeur imp. Paris 1933' :'imprimeur',
        'imprimeur colin paul'      :'imprimeur',
        'imprimerie phototypique'   :'imprimeur phototypique',
        'imprimeure photographique' :'imprimeur photographique',
        'imprimeur ã\xa0 marseille' :'imprimeur',
        'imprimeur paris'           :'imprimeur',
        'phototypie ,imprimeur'     :'imprimeur',
        'imprmeur'                  :'imprimeur',
        'imprimeur-libraire'        :'imprimeur, libraire',
        'imprimeur 53 rue rochechouart paris':'imprimeur',
        'dit(dessinateur'           :'dessinateur',
        'fondeurs'                  :'fondeur',
        'dit(inventeur'             :'inventeur',
        'dessinateur  creation imp' :'dessinateur',
        'dessinateur,cartonnier'    :'dessinateur, cartonnier',
        'dit(peintre'               :'peintre',
        '?,peintre'                 :'peintre',
        'décoratrice'               :'décorateur',
        'artiste-décorateur'        :'décorateur',
        'décorateur?'               :'décorateur',
        'imprimeur comite national de defense contre la tuberculose':'imprimeur',
        'architecte(s et sculpteur(s':'architecte, sculpteur',
        'maitre-'                   :"maître-",
        'maitre '                   :"maître-",
        'maître- '                  :"maître-",
        'facteur de clavecins'      :'facteur',
        '?,'                        :'',
        '(?'                        :'',
        ',?'                        :'',
        'eleve'                     :'élève',
    }

temp.loc[temp[2]==temp[1], 2] = np.nan

# 557 rows × 3 columns
for col in [1,2,3]:
    # - 6 rows
    temp.loc[(temp[col].notna()), col] = temp.loc[(temp[col].notna()), col].str.replace(")", "", regex=False)
    temp.loc[(temp[col].notna()), col] = temp.loc[(temp[col].notna()), col].str.replace("?", "", regex=False)
    temp.loc[(temp[col].notna()), col] = temp.loc[(temp[col].notna()), col].str.replace(r"^,", "", regex=True)
    temp.loc[(temp[col].notna()), col] = temp.loc[(temp[col].notna()), col].str.strip()

    temp.loc[(temp[col].notna()), col] = temp.loc[(temp[col].notna()), col].str.lower()

    for str_ in to_replace_nan:
        temp.loc[temp[col]==str_.strip().lower(), col] = np.nan
        
    # Suppression de toutes les dates
    # "1781-1864", '1852-…', '1837-1913', '1834-1916','1830-1884','1753-1823','1879-1946', '1791-1849', '1720-1789','1805-1863',
    # '1810-1887', '1739-1814', '1847-1914','1858 -1941', '1767-1855','1649-1712','1866-1937','1757-1836','1856- 1956','1834-1899',
    temp.loc[temp[col].notna(), col] = temp.loc[temp[col].notna(), col].str.replace("^\d{4}-\d{4}$", "", regex=True)
    
    for str_ in to_replace_nan_contains:
        temp.loc[(temp[col].notna()) & (temp[col].str.contains(str_.strip().lower(), regex=False)), col] = np.nan
            
    for str_, str_2 in to_replace_by.items():
        temp.loc[temp[col].notna(), col] = temp.loc[temp[1].notna(), col].str.replace(str_.strip().lower(), str_2.strip().lower(), regex=False)

    temp.loc[(temp[col].notna()) & (temp[col]==""), col] = np.nan
# 431 rows × 3 columns

In [408]:
temp[1] = temp[1].fillna(temp[2])
temp[1] = temp[1].fillna(temp[3])
temp.loc[temp[2]==temp[1], 2] = np.nan
temp.loc[temp[3]==temp[1], 3] = np.nan

In [411]:
temp[(temp[1].notna()) & (temp[2].notna())].head(20)

,0,1,2,3
178,VALENS,flavius valens,émetteur,None
892,DONADIER,couturier,couturier,None
1008,VALENTINIEN IER,flavius valentinianus,émetteur,None
1467,FAIENCERIES DE CREIL & MONTEREAU,lantz,auteur du décor,None
2060,VALENTINIEN III,placidius valentinianus,émetteur,None
2982,CONSTANT IER,flavius julius constans,émetteur,None
2983,MAXIME,magnus clemens maximus,émetteur,None
4411,GREBER Johann Peter,céramiste,auteur du modèle,None
4415,FAIENCERIES DE CREIL & MONTEREAU,froment richard antoine albert,auteur du décor,None
4909,XUANZONG,qing,émetteur,None


In [386]:
temp[(temp[1].notna()) & (temp[2].notna())].tail(20)

,0,1,2
117589,MAXIME PUPIEN,dit marcus clodius pupienus maximus,émetteur
117985,NADAR,dit,photographe
119323,CLESINGER Jean-Baptiste,dit auguste,sculpteur
119627,JOUET DE PARIS,jdep et jep,fabricant
119723,BREGER A,frères,éditeur
119895,RABIER-DONOT JEAN,j.r.d.,fabricant
119959,RICHON-BRUNET,dit,peintre
120705,Institut géographique national,france,auteur
120780,SIMONIN - CUNY,s.c.,fabricant
122330,PONTORMO IL,dit carrucci jacopo,auteur


In [414]:
tp2 = temp[2].value_counts().to_dict()
tp1 = temp[1].value_counts().to_dict()
tp3 = temp[3].value_counts().to_dict()
counter_final = Counter(tp2) + Counter(tp1) + Counter(tp3)
counter_final.most_common()[-200:-1]

[('imprimeur, libraire', 2),
 ('réducteur', 2),
 ('manière', 2),
 ('marbrerie', 2),
 ('entrepreneur', 2),
 ('emaillographe', 2),
 ('compassier-poullieur', 2),
 ('auguste divinisé', 2),
 ('peintre amateur', 2),
 ('facteur de pianos', 2),
 ('tourneur sur bois', 2),
 ('poterie', 2),
 ('administrateur', 2),
 ('vernisseur', 2),
 ('serrurier', 2),
 ('londres', 2),
 ('france', 2),
 ('pseudonyme', 2),
 ('attribution supposée', 2),
 ('jules', 2),
 (',graveur', 2),
 ('besançon', 2),
 ('suiveur de', 2),
 ('artiste', 2),
 ('auteur de la traduction', 2),
 ('bottier', 2),
 ('graphiste', 2),
 ('archetier', 2),
 ('del', 2),
 ('ebéniste', 2),
 ('adaptateur', 2),
 ('joailler', 2),
 ('diagraphe', 2),
 ('manufacturier', 2),
 ('brunisseur', 2),
 ('pantographe', 2),
 ('cadranier', 2),
 ('dinandier', 2),
 ('éventailliste', 2),
 ('brodeur', 2),
 ('hydrographe', 2),
 ('astronome', 2),
 ('facteur', 2),
 ('flavius valentinianus', 2),
 ('fabrique de vêtements liturgiques', 2),
 ('ferblantier', 2),
 ('ethnie', 2),

In [416]:
from collections import defaultdict

In [420]:
nb_val_dict = defaultdict(list)

for key, value in counter_final.items():
    nb_val_dict[value].append(key)

order = sorted(list(nb_val_dict.keys()))
buffer = ""
for nb in order:
    metiers = sorted(nb_val_dict[nb])
    for m in metiers:
        buffer += f"{m.strip()};{nb};\n"
print(buffer)

lunetier;1;
& son fils f.;1;
1852-…;1;
1856- 1956;1;
1858 -1941;1;
ad.;1;
antonia;1;
antonin le pieux divinisé;1;
architecte, sculpteur;1;
architecte-concepteur;1;
archéologue;1;
atelier,;1;
auteur de costumes;1;
auteur de la compositeur;1;
auteur de la patine;1;
auteur des ornements;1;
beaucourt;1;
bernard;1;
bracquemond félix;1;
bromure;1;
buraliste;1;
buronnier;1;
c.abel-klinger;1;
c.r;1;
caecilius saecularis;1;
charpentier de marine;1;
charron;1;
cloutier;1;
combier;1;
commerçant;1;
compassier;1;
cordonnier;1;
culture;1;
céramistes;1;
de jacob;1;
de jean-dieu;1;
de jean-jacques;1;
de pierre;1;
dentelier;1;
dessinateur, cartonnier;1;
diagrape;1;
dom lorenzo;1;
drusus;1;
décorateur de porcelaine;1;
emailleur;1;
eug.;1;
faustine ii;1;
faustine mère;1;
fils;1;
fils pierre;1;
flavius claudius julianus;1;
flavius julius constans;1;
flavius valens;1;
françois;1;
fratelli;1;
froment richard antoine albert;1;
frères;1;
g.s.;1;
giandomenico;1;
graveur en médaille;1;
graveur sur bois;1;
grave

In [400]:
temp.shape

(70115, 3)

In [381]:
temp.loc[temp[2]=='?)', 2] = np.nan

In [382]:
df_auteurs['complement'] = df_auteurs[0].str.split("(")
df_auteurs.head()

KeyError: 0

<div style="display: flex; background-color: blue;" >

# ANNEXE
</div>

<div style="display: flex; background-color: green;" >

## 2.1. Titre 2
</div>

<div style="display: flex; background-color: indigo;" >

#### Titre 3
</div>

<div style="display: flex; background-color: deeppink;" >

##### Titre4
</div>

<div style="display: flex; background-color: coral;" >

##### Titre 5
</div>